In [ ]:
%load_ext autoreload
%autoreload 2

# Test Workflow Data Aggregation

-----------------------------------------------------------------
This example illustrates the aggregation of workflow (actions) data using the `github2pandas` repository. Here, all workflows are read out, filtered and displayed in terms of success or failure. 

In [ ]:
from github2pandas.aggregation.workflows import AggWorkflow as AggWF
from github2pandas.aggregation.utility import Utility
from pathlib import Path

## Basic Usage

The most important input parameter is an Repository object from PyGitHub-Package.

In [ ]:
git_repo_name = "Extract_Git_Activities"
git_repo_owner = "TUBAF-IFI-DiPiT"
    
default_data_folder = Path("data", git_repo_name)

import os
github_token = os.environ['TOKEN']
# If you do not include your Github Token in .env, its neccessary to integrate it here. 
# github_token = "yourToken"

repo = Utility.get_repo(git_repo_owner, git_repo_name, github_token)

The code snipet generates a raw data set based on repo information. The pandas Dataframe includes author´s information, timestamp and the general result of the workflow run. 

In [ ]:
AggWF.generate_workflow_pandas_tables(repo=repo, data_root_dir=default_data_folder)

In [ ]:
pd_workflow = AggWF.get_raw_workflow(data_root_dir=default_data_folder)
pd_workflow.head(5)

In [ ]:
pd_run = AggWF.get_raw_run(data_root_dir=default_data_folder)
pd_run.head(5)

## Get workflow run logs
What happens during the workflow run. Let's take a closer view on log files of a specific Action run.

In [ ]:
AggWF.download_workflow_log_files(repo=repo,
                                  github_token=github_token,
                                  run = repo.get_workflow_run(642018321),
                                  data_root_dir=default_data_folder)